## This notebook is for experimenting with AO2MO algorithms

In [1]:
import pyscf
from pyscf import mp, mcscf
import numpy as np
import h5py
import time
import scipy
import itertools
import copy
import matplotlib.pyplot as plt
import os, glob

MACHEPS = 1e-9
EH_TO_WN = 219474.63
EH_TO_EV = 27.211399

def eri_h5_write(mol, mo_coeffs, intor, erifile='tmp.h5', gaunt=False, terminal=False):
    phase = -1 if gaunt else 1
    pyscf.ao2mo.r_outcore.general(mol, mo_coeffs, erifile=erifile, dataname='tmp', intor=intor, aosym='s1')
    blksize = 400
    nij = mo_coeffs[0].shape[1] * mo_coeffs[1].shape[1]
    with h5py.File(erifile, mode='r+') as feri:
        for i0, i1 in pyscf.lib.prange(0, nij, blksize):
            buf = feri['mo_eri'][i0:i1]
            buf += phase*feri['tmp'][i0:i1]
            feri['mo_eri'][i0:i1] = buf
        
        if (terminal):
            del feri['tmp']

In [2]:
class RelForte:
    def __init__(self, mol, c0=None, verbose=True, density_fitting=False, decontract=False):
        if (type(density_fitting) is bool):
            self.density_fitting = density_fitting
            self.df_basis = None
        elif(type(density_fitting) is str):
            self.density_fitting = True
            self.df_basis = density_fitting
        self.decontract = decontract
        if (self.decontract):
            self.mol, _ = mol.decontract_basis()
        else:
            self.mol = mol
        self.nuclear_repulsion = self.mol.energy_nuc()
        self.nelec = sum(self.mol.nelec)
        self.nocc = self.nelec
        self.nao = self.mol.nao
        self.nlrg = self.mol.nao*2
        self.nvirtual = self.nlrg - self.nocc
        self.verbose = verbose
        if (c0 is None):
            self.c0 = pyscf.lib.param.LIGHT_SPEED
        else:
            self.c0 = c0
            #pyscf.lib.param.LIGHT_SPEED = c0
        
    def run_dhf(self, transform=False, debug=False, frozen=None, with_gaunt=False, with_breit=False, with_ssss=True, dump_mo_coeff=None, algo='disk', erifile=None, fake_dhf=None):
        _ti = time.time()

        if (self.verbose):
            print('='*47)
            print('{:^47}'.format('PySCF DHF interface'))
            print('='*47)

        # Run relativistic Dirac-Hartree-Fock
        if (self.density_fitting):
            if (self.verbose): print('{:#^47}'.format('Enabling density fitting!')) 
            self.dhf = pyscf.scf.DHF(self.mol).density_fit() if self.df_basis is None else pyscf.scf.DHF(self.mol).density_fit(self.df_basis)
        else:
            self.dhf = pyscf.scf.DHF(self.mol)

        self.dhf.with_gaunt = with_gaunt
        self.dhf.with_breit = with_breit
        self.dhf.with_ssss = with_ssss
        
        if (type(fake_dhf) is str):
            f = h5py.File(fake_dhf, 'r')
            self.dhf_energy = f['SCF']['TOTAL_ENERGY'][:][0]

        else:
            self.dhf_energy = self.dhf.kernel()
            if (self.verbose): 
                _t0 = time.time()
                print(f"Relativistic DHF Energy:     {self.dhf_energy:15.7f} Eh")
                print(f'PySCF RHF time:              {_t0-_ti:15.7f} s')
                print('-'*47)

            if (dump_mo_coeff is not None):
                if dump_mo_coeff != False:
                    if type(dump_mo_coeff) is str:
                        _fname = dump_mo_coeff
                    else:
                        _fname = 'mo_coeff'
                    if (self.verbose): print(f'Dumping MO coefficients to {_fname}')
                    np.save(_fname, self.dhf.mo_coeff)

            if (transform):
                self.rel_ao2mo(self.dhf.mo_coeff, frozen, algo, erifile)
                
                self.dhf_e1 = np.einsum('ii->',self.dhf_hcore_mo[:self.nocc, :self.nocc])
                self.dhf_e2 = 0.5*np.einsum('ijij->',self.dhf_eri_full_asym[:self.nocc, :self.nocc, :self.nocc, :self.nocc])            
                self.dhf_e_rebuilt = self.dhf_e1 + self.dhf_e2 + self.nuclear_repulsion
                if (debug and self.verbose):
                    print(f"Rebuilt DHF Energy:          {self.dhf_e_rebuilt.real:15.7f} Eh")
                    print(f"Error to PySCF:              {np.abs(self.dhf_e_rebuilt.real - self.dhf_energy):15.7f} Eh")
                    if (frozen is None):
                        print(f"Diff 1e:                     {np.abs(self.dhf.scf_summary['e1']-self.dhf_e1):15.7f} Eh")
                        print(f"Diff 2e:                     {np.abs(self.dhf.scf_summary['e2']-self.dhf_e2):15.7f} Eh")
                
                _t3 = time.time()
                
                if (self.verbose):
                    print(f'Total time taken:            {(_t3-_t0):15.7f} s')
                    print('='*47)

    def ao2mo_einsum(self, mo_coeff, nlrg, eri, moslice):
        _mo_l = mo_coeff[:nlrg, nlrg:]
        _mo_s = mo_coeff[nlrg:, nlrg:]

        # LLLL
        eri += np.einsum('pi,qj,pqrs,rk,sl->ijkl', np.conj(_mo_l[:,moslice[0]]),(_mo_l[:,moslice[1]]),self.mol.intor('int2e_spinor'),np.conj(_mo_l[:,moslice[2]]),(_mo_l[:,moslice[3]]),optimize=True)
        # SSSS
        eri += np.einsum('pi,qj,pqrs,rk,sl->ijkl', np.conj(_mo_s[:,moslice[0]]),(_mo_s[:,moslice[1]]),self.mol.intor('int2e_spsp1spsp2_spinor'),np.conj(_mo_s[:,moslice[2]]),(_mo_s[:,moslice[3]]),optimize=True)/(2*self.c0)**4
        # SSLL
        eri += np.einsum('pi,qj,pqrs,rk,sl->ijkl', np.conj(_mo_s[:,moslice[0]]),(_mo_s[:,moslice[1]]),self.mol.intor('int2e_spsp1_spinor'),np.conj(_mo_l[:,moslice[2]]),(_mo_l[:,moslice[3]]),optimize=True)/(2*self.c0)**2
        # LLSS
        eri += np.einsum('pi,qj,pqrs,rk,sl->ijkl', np.conj(_mo_l[:,moslice[0]]),(_mo_l[:,moslice[1]]),self.mol.intor('int2e_spsp2_spinor'),np.conj(_mo_s[:,moslice[2]]),(_mo_s[:,moslice[3]]),optimize=True)/(2*self.c0)**2

        if (self.dhf.with_breit):
            # LSLS
            eri += np.einsum('pi,qj,pqrs,rk,sl->ijkl', np.conj(_mo_l[:,moslice[0]]),(_mo_s[:,moslice[1]]),self.mol.intor('int2e_breit_ssp1ssp2_spinor'),np.conj(_mo_l[:,moslice[2]]),(_mo_s[:,moslice[3]]),optimize=True)/(2*self.c0)**2
            # LSSL
            eri += np.einsum('pi,qj,pqrs,rk,sl->ijkl', np.conj(_mo_l[:,moslice[0]]),(_mo_s[:,moslice[1]]),self.mol.intor('int2e_breit_ssp1sps2_spinor'),np.conj(_mo_s[:,moslice[2]]),(_mo_l[:,moslice[3]]),optimize=True)/(2*self.c0)**2
            # SLLS
            eri += np.einsum('pi,qj,pqrs,rk,sl->ijkl', np.conj(_mo_s[:,moslice[0]]),(_mo_l[:,moslice[1]]),self.mol.intor('int2e_breit_sps1ssp2_spinor'),np.conj(_mo_l[:,moslice[2]]),(_mo_s[:,moslice[3]]),optimize=True)/(2*self.c0)**2
            # SLSL
            eri += np.einsum('pi,qj,pqrs,rk,sl->ijkl', np.conj(_mo_s[:,moslice[0]]),(_mo_l[:,moslice[1]]),self.mol.intor('int2e_breit_sps1sps2_spinor'),np.conj(_mo_s[:,moslice[2]]),(_mo_l[:,moslice[3]]),optimize=True)/(2*self.c0)**2                        
        elif (self.dhf.with_gaunt):
            # Gaunt term doesn't account for the negative sign, whereas the Breit one does
            # LSLS
            eri -= np.einsum('pi,qj,pqrs,rk,sl->ijkl', np.conj(_mo_l[:,moslice[0]]),(_mo_s[:,moslice[1]]),self.mol.intor('int2e_ssp1ssp2_spinor'),np.conj(_mo_l[:,moslice[2]]),(_mo_s[:,moslice[3]]),optimize=True)/(2*self.c0)**2
            # LSSL
            eri -= np.einsum('pi,qj,pqrs,rk,sl->ijkl', np.conj(_mo_l[:,moslice[0]]),(_mo_s[:,moslice[1]]),self.mol.intor('int2e_ssp1sps2_spinor'),np.conj(_mo_s[:,moslice[2]]),(_mo_l[:,moslice[3]]),optimize=True)/(2*self.c0)**2
            # SLLS
            eri -= np.einsum('pi,qj,pqrs,rk,sl->ijkl', np.conj(_mo_s[:,moslice[0]]),(_mo_l[:,moslice[1]]),self.mol.intor('int2e_sps1ssp2_spinor'),np.conj(_mo_l[:,moslice[2]]),(_mo_s[:,moslice[3]]),optimize=True)/(2*self.c0)**2
            # SLSL
            eri -= np.einsum('pi,qj,pqrs,rk,sl->ijkl', np.conj(_mo_s[:,moslice[0]]),(_mo_l[:,moslice[1]]),self.mol.intor('int2e_sps1sps2_spinor'),np.conj(_mo_s[:,moslice[2]]),(_mo_l[:,moslice[3]]),optimize=True)/(2*self.c0)**2                        
        return eri


    def rel_ao2mo(self, mo_coeff, frozen, algo='disk', erifile=None, eriread=None):
        def h5_write(fname, slices):
            pyscf.ao2mo.r_outcore.general(self.mol, (_mo_l[:,slices[0]], _mo_l[:,slices[1]], _mo_l[:,slices[2]], _mo_l[:,slices[3]]), erifile=fname, dataname='mo_eri', intor='int2e_spinor',aosym='s1')
            eri_h5_write(self.mol, (_mo_s[:,slices[0]],_mo_s[:,slices[1]],_mo_s[:,slices[2]],_mo_s[:,slices[3]]), 'int2e_spsp1spsp2_spinor', erifile=fname)
            eri_h5_write(self.mol, (_mo_s[:,slices[0]],_mo_s[:,slices[1]],_mo_l[:,slices[2]],_mo_l[:,slices[3]]), 'int2e_spsp1_spinor', erifile=fname)
            eri_h5_write(self.mol, (_mo_l[:,slices[0]],_mo_l[:,slices[1]],_mo_s[:,slices[2]],_mo_s[:,slices[3]]), 'int2e_spsp2_spinor', erifile=fname, terminal=(not (self.dhf.with_breit or self.dhf.with_gaunt)))
            if (self.dhf.with_breit or self.dhf.with_gaunt):
                if (self.dhf.with_gaunt):
                    eri_h5_write(self.mol, (_mo_l[:,slices[0]],_mo_s[:,slices[1]],_mo_l[:,slices[2]],_mo_s[:,slices[3]]), 'int2e_ssp1ssp2_spinor', erifile=fname, gaunt=True)
                    eri_h5_write(self.mol, (_mo_l[:,slices[0]],_mo_s[:,slices[1]],_mo_s[:,slices[2]],_mo_l[:,slices[3]]), 'int2e_ssp1sps2_spinor', erifile=fname, gaunt=True)
                    eri_h5_write(self.mol, (_mo_s[:,slices[0]],_mo_l[:,slices[1]],_mo_l[:,slices[2]],_mo_s[:,slices[3]]), 'int2e_sps1ssp2_spinor', erifile=fname, gaunt=True)
                    eri_h5_write(self.mol, (_mo_s[:,slices[0]],_mo_l[:,slices[1]],_mo_s[:,slices[2]],_mo_l[:,slices[3]]), 'int2e_sps1sps2_spinor', erifile=fname, gaunt=True, terminal=True)
                else:
                    eri_h5_write(self.mol, (_mo_l[:,slices[0]],_mo_s[:,slices[1]],_mo_l[:,slices[2]],_mo_s[:,slices[3]]), 'int2e_breit_ssp1ssp2_spinor', erifile=fname)
                    eri_h5_write(self.mol, (_mo_l[:,slices[0]],_mo_s[:,slices[1]],_mo_s[:,slices[2]],_mo_l[:,slices[3]]), 'int2e_breit_ssp1sps2_spinor', erifile=fname)
                    eri_h5_write(self.mol, (_mo_s[:,slices[0]],_mo_l[:,slices[1]],_mo_l[:,slices[2]],_mo_s[:,slices[3]]), 'int2e_breit_sps1ssp2_spinor', erifile=fname)
                    eri_h5_write(self.mol, (_mo_s[:,slices[0]],_mo_l[:,slices[1]],_mo_s[:,slices[2]],_mo_l[:,slices[3]]), 'int2e_breit_sps1sps2_spinor', erifile=fname, terminal=True)
                    
        _t0 = time.time()
        self.norb = self.mol.nao_2c()*2

        # Harvest h from DHF (Includes both S & L blocks)
        _dhf_hcore_ao = self.dhf.get_hcore()
        _dhf_hcore_mo = np.einsum('pi,pq,qj->ij', np.conj(mo_coeff[:,self.nlrg:]), _dhf_hcore_ao, mo_coeff[:,self.nlrg:])

        del _dhf_hcore_ao

        if (frozen == 0 or frozen == (0,0)):
            frozen = None

        if (frozen is not None):
            try:
                assert (type(frozen) is int or type(frozen) is tuple) 
                if (type(frozen) is int):
                    assert (frozen >= 0 and frozen < self.nelec)
                else:
                    assert ((frozen[0] >= 0 and frozen[0] < self.nelec) and frozen[1] <= self.nvirtual)
            except AssertionError:
                raise Exception("The 'frozen' argument must be an integer or tuple of integers, and they have to fit into the spectrum!")
            
            
            if (type(frozen) is int):
                self.nfrozen = frozen
                self.nfrozen_virt = 0
            else:
                self.nfrozen = frozen[0]
                self.nfrozen_virt = frozen[1]
            _nlrg = self.nlrg # We need to preserve the original nlrg for just a little bit longer..
            self.nlrg -= (self.nfrozen + self.nfrozen_virt)
            
            self.nelec -= self.nfrozen
            self.nocc -= self.nfrozen
            self.nvirtual -= self.nfrozen_virt

            _frzc = slice(0,self.nfrozen)
            _actv = slice(self.nfrozen,self.nlrg+self.nfrozen)

            self.e_frozen = np.einsum('ii->',_dhf_hcore_mo[_frzc,_frzc]) # The 1e part is common to both with DF and without, the 2e part is done later
        else:
            self.nfrozen = 0
            self.e_frozen = 0.0
            self.dhf_hcore_mo = _dhf_hcore_mo
            _frzc = slice(0,self.nfrozen)
            _actv = slice(self.nfrozen,self.nlrg+self.nfrozen)
            _nlrg = self.nlrg

        if (self.density_fitting):
            self.naux = self.dhf.with_df._cderi[0].shape[0]
            _mem = 2*(self.naux*_nlrg**2)*16/1e9
            if (_mem < 1.0):
                if (self.verbose): print(f'Will now allocate {_mem*1000:.3f} MB memory for the DF AO ERI tensor!')
            else:
                if (self.verbose): print(f'Will now allocate {_mem:.3f} GB memory for the DF AO ERI tensor!')

            _Lpq_LL = pyscf.lib.unpack_tril(self.dhf.with_df._cderi[0]) # 0 is eri_LL, 1 is eri_SS
            _Lpq_SS = pyscf.lib.unpack_tril(self.dhf.with_df._cderi[1])/(2*pyscf.lib.param.LIGHT_SPEED)**2 # 0 is eri_LL, 1 is eri_SS

            _Lpq_mo_LL = np.einsum('ip,jq,lij->lpq',np.conj(mo_coeff[:_nlrg,_nlrg:]),mo_coeff[:_nlrg,_nlrg:],_Lpq_LL,optimize='optimal')
            _Lpq_mo_SS = np.einsum('ip,jq,lij->lpq',np.conj(mo_coeff[_nlrg:,_nlrg:]),mo_coeff[_nlrg:,_nlrg:],_Lpq_SS,optimize='optimal')
            del _Lpq_LL, _Lpq_SS

            _t1 = time.time()

            if (frozen is not None):
                # The 2e part of e_frozen
                self.e_frozen += 0.5*(np.einsum('lii,ljj->',_Lpq_mo_LL[:,_frzc,_frzc],_Lpq_mo_LL[:,_frzc,_frzc])+np.einsum('lii,ljj->',_Lpq_mo_SS[:,_frzc,_frzc],_Lpq_mo_SS[:,_frzc,_frzc])+np.einsum('lii,ljj->',_Lpq_mo_SS[:,_frzc,_frzc],_Lpq_mo_LL[:,_frzc,_frzc])+np.einsum('lii,ljj->',_Lpq_mo_LL[:,_frzc,_frzc],_Lpq_mo_SS[:,_frzc,_frzc]))
                self.e_frozen -= 0.5*(np.einsum('lij,lji->',_Lpq_mo_LL[:,_frzc,_frzc],_Lpq_mo_LL[:,_frzc,_frzc])+np.einsum('lij,lji->',_Lpq_mo_SS[:,_frzc,_frzc],_Lpq_mo_SS[:,_frzc,_frzc])+np.einsum('lij,lji->',_Lpq_mo_SS[:,_frzc,_frzc],_Lpq_mo_LL[:,_frzc,_frzc])+np.einsum('lij,lji->',_Lpq_mo_LL[:,_frzc,_frzc],_Lpq_mo_SS[:,_frzc,_frzc]))
                
                self.dhf_hcore_mo = _dhf_hcore_mo[_actv,_actv].copy()
                del _dhf_hcore_mo
                self.dhf_hcore_mo += np.einsum('lpq,lii->pq',_Lpq_mo_LL[:,_actv,_actv],_Lpq_mo_LL[:,:self.nfrozen,:self.nfrozen])+np.einsum('lpq,lii->pq',_Lpq_mo_SS[:,_actv,_actv],_Lpq_mo_SS[:,:self.nfrozen,:self.nfrozen])+np.einsum('lpq,lii->pq',_Lpq_mo_SS[:,_actv,_actv],_Lpq_mo_LL[:,:self.nfrozen,:self.nfrozen])+np.einsum('lpq,lii->pq',_Lpq_mo_LL[:,_actv,_actv],_Lpq_mo_SS[:,:self.nfrozen,:self.nfrozen])
                self.dhf_hcore_mo -= np.einsum('lpi,liq->pq',_Lpq_mo_LL[:,_actv,_frzc],_Lpq_mo_LL[:,_frzc,_actv])+np.einsum('lpi,liq->pq',_Lpq_mo_SS[:,_actv,_frzc],_Lpq_mo_SS[:,_frzc,_actv])+np.einsum('lpi,liq->pq',_Lpq_mo_SS[:,_actv,_frzc],_Lpq_mo_LL[:,_frzc,_actv])+np.einsum('lpi,liq->pq',_Lpq_mo_LL[:,_actv,_frzc],_Lpq_mo_SS[:,_frzc,_actv])


            _mem = (self.nlrg**4)*16/1e9
            if (_mem < 1.0):
                if (self.verbose): print(f'Will now allocate {_mem*1000:.3f} MB memory for the MO ERI tensor!')
            else:
                if (self.verbose): print(f'Will now allocate {_mem:.3f} GB memory for the MO ERI tensor!')

            self.dhf_eri_full_asym = np.einsum('lpq,lrs->pqrs',_Lpq_mo_LL[:,_actv,_actv],_Lpq_mo_LL[:,_actv,_actv],optimize='optimal') + np.einsum('lpq,lrs->pqrs',_Lpq_mo_SS[:,_actv,_actv],_Lpq_mo_SS[:,_actv,_actv],optimize='optimal') + np.einsum('lpq,lrs->pqrs',_Lpq_mo_LL[:,_actv,_actv],_Lpq_mo_SS[:,_actv,_actv],optimize='optimal') + np.einsum('lpq,lrs->pqrs',_Lpq_mo_SS[:,_actv,_actv],_Lpq_mo_LL[:,_actv,_actv],optimize='optimal')
            del _Lpq_mo_LL, _Lpq_mo_SS
            self.dhf_eri_full_asym = self.dhf_eri_full_asym.swapaxes(1,2) - self.dhf_eri_full_asym.swapaxes(1,2).swapaxes(2,3)
            _t2 = time.time()
        else:
            if (algo == 'direct'):

                if (frozen is not None):
                    _mem = self.nfrozen**4*16/1e9*2
                    if (_mem < 1.0):
                        if (self.verbose): print(f'Will now allocate {_mem*1000:.3f} MB memory for the frozen core ERI tensor!')
                    else:
                        if (self.verbose): print(f'Will now allocate {_mem:.3f} GB memory for the frozen core ERI tensor!')

                    eri_fc_ao = np.zeros((self.nfrozen,self.nfrozen,self.nfrozen,self.nfrozen),dtype='complex128')
                    eri_fc = self.ao2mo_einsum(mo_coeff, _nlrg, eri_fc_ao, (_frzc,_frzc,_frzc,_frzc))
                    self.e_frozen += 0.5*(np.einsum('iijj->',eri_fc)-np.einsum('ijji->',eri_fc))
                    del eri_fc, eri_fc_ao

                    _mem = 2*(self.nlrg*self.nfrozen)**2*16/1e9*2
                    if (_mem < 1.0):
                        if (self.verbose): print(f'Will now allocate {_mem*1000:.3f} MB memory for the aacc and acca ERI tensor!')
                    else:
                        if (self.verbose): print(f'Will now allocate {_mem:.3f} GB memory for the aacc and acca ERI tensor!')

                    eri_aacc_ao = np.zeros((self.nlrg,self.nlrg,self.nfrozen,self.nfrozen),dtype='complex128')
                    eri_aacc = self.ao2mo_einsum(mo_coeff, _nlrg, eri_aacc_ao, (_actv,_actv,_frzc,_frzc))
                    self.dhf_hcore_mo = _dhf_hcore_mo[_actv,_actv].copy()
                    self.dhf_hcore_mo += np.einsum('pqii->pq', eri_aacc)
                    del eri_aacc_ao, eri_aacc

                    eri_acca_ao = np.zeros((self.nlrg,self.nfrozen,self.nfrozen,self.nlrg),dtype='complex128')
                    eri_acca = self.ao2mo_einsum(mo_coeff, _nlrg, eri_acca_ao, (_actv,_frzc,_frzc,_actv))
                    self.dhf_hcore_mo -= np.einsum('piiq->pq', eri_acca)
                    del eri_acca_ao, eri_acca


                _mem = _nlrg**4*16/1e9
                if (_mem < 1.0):
                    if (self.verbose): print(f'Will now allocate {_mem*1000:.3f} MB memory for the AO ERI tensor!')
                else:
                    if (self.verbose): print(f'Will now allocate {_mem:.3f} GB memory for the AO ERI tensor!')

                _dhf_eri_mo_full = np.zeros((self.nlrg,self.nlrg,self.nlrg,self.nlrg),dtype='complex128')
                _dhf_eri_mo_full = self.ao2mo_einsum(mo_coeff, _nlrg, _dhf_eri_mo_full, (_actv,_actv,_actv,_actv))

                _t1 = time.time()
                
                _dhf_eri_full_asym = _dhf_eri_mo_full.swapaxes(1,2) - _dhf_eri_mo_full.swapaxes(1,2).swapaxes(2,3)
        
                del _dhf_eri_mo_full

                _t2 = time.time()

                if (frozen is not None):
                    self.dhf_eri_full_asym = _dhf_eri_full_asym
                    del _dhf_eri_full_asym
                else:
                    self.e_frozen = 0.0
                    self.dhf_eri_full_asym = _dhf_eri_full_asym
                    self.dhf_hcore_mo = _dhf_hcore_mo
            
            elif (algo == 'naive'):
                _mem = self.norb**4*16/1e9
                if (_mem < 1.0):
                    if (self.verbose): print(f'Will now allocate {_mem*1000:.3f} MB memory for the AO ERI tensor!')
                else:
                    if (self.verbose): print(f'Will now allocate {_mem:.3f} GB memory for the AO ERI tensor!')

                _dhf_eri_ao = np.zeros((self.norb,self.norb,self.norb,self.norb),dtype='complex128')
                _dhf_eri_ao[:_nlrg,:_nlrg,:_nlrg,:_nlrg] = self.mol.intor('int2e_spinor')
                _dhf_eri_ao[_nlrg:,_nlrg:,_nlrg:,_nlrg:] = self.mol.intor('int2e_spsp1spsp2_spinor')/(2*self.c0)**4
                _dhf_eri_ao[_nlrg:,_nlrg:,:_nlrg,:_nlrg] = self.mol.intor('int2e_spsp1_spinor')/(2*self.c0)**2
                _dhf_eri_ao[:_nlrg,:_nlrg,_nlrg:,_nlrg:] = self.mol.intor('int2e_spsp2_spinor')/(2*self.c0)**2
                if (self.dhf.with_breit or self.dhf.with_gaunt):
                    if (self.dhf.with_breit):
                        _dhf_eri_ao[:_nlrg,_nlrg:,:_nlrg,_nlrg:] = self.mol.intor('int2e_breit_ssp1ssp2_spinor')/(2*self.c0)**2
                        _dhf_eri_ao[:_nlrg,_nlrg:,_nlrg:,:_nlrg] = self.mol.intor('int2e_breit_ssp1sps2_spinor')/(2*self.c0)**2
                        _dhf_eri_ao[_nlrg:,:_nlrg,:_nlrg,_nlrg:] = self.mol.intor('int2e_breit_sps1ssp2_spinor')/(2*self.c0)**2
                        _dhf_eri_ao[_nlrg:,:_nlrg,_nlrg:,:_nlrg] = self.mol.intor('int2e_breit_sps1sps2_spinor')/(2*self.c0)**2
                    else:
                        # Gaunt term doesn't account for the negative sign, whereas the Breit one does
                        _dhf_eri_ao[:_nlrg,_nlrg:,:_nlrg,_nlrg:] = -self.mol.intor('int2e_ssp1ssp2_spinor')/(2*self.c0)**2
                        _dhf_eri_ao[:_nlrg,_nlrg:,_nlrg:,:_nlrg] = -self.mol.intor('int2e_ssp1sps2_spinor')/(2*self.c0)**2
                        _dhf_eri_ao[_nlrg:,:_nlrg,:_nlrg,_nlrg:] = -self.mol.intor('int2e_sps1ssp2_spinor')/(2*self.c0)**2
                        _dhf_eri_ao[_nlrg:,:_nlrg,_nlrg:,:_nlrg] = -self.mol.intor('int2e_sps1sps2_spinor')/(2*self.c0)**2


                _t1 = time.time()
                
                _dhf_eri_mo_full = np.einsum('pi,qj,pqrs,rk,sl->ijkl',np.conj(mo_coeff[:,_nlrg:]),(mo_coeff[:,_nlrg:]),_dhf_eri_ao,np.conj(mo_coeff[:,_nlrg:]),(mo_coeff[:,_nlrg:]),optimize=True)
                _dhf_eri_full_asym = _dhf_eri_mo_full.swapaxes(1,2) - _dhf_eri_mo_full.swapaxes(1,2).swapaxes(2,3)
        
                del _dhf_eri_ao, _dhf_eri_mo_full

                _t2 = time.time()

                if (frozen is not None):
                    self.e_frozen += 0.5*np.einsum('ijij->',_dhf_eri_full_asym[:self.nfrozen,:self.nfrozen,:self.nfrozen,:self.nfrozen])

                    self.dhf_hcore_mo = _dhf_hcore_mo[_actv, _actv].copy() + np.einsum('ipjp->ij',_dhf_eri_full_asym[_actv,_frzc,_actv,_frzc])
                    del _dhf_hcore_mo
                    self.dhf_eri_full_asym = _dhf_eri_full_asym[_actv,_actv,_actv,_actv]
                    del _dhf_eri_full_asym
                else:
                    self.e_frozen = 0.0
                    self.dhf_eri_full_asym = _dhf_eri_full_asym
                    self.dhf_hcore_mo = _dhf_hcore_mo
            elif (algo == 'disk'):
                _cleanup = True
                if (erifile is None or type(erifile) is not str):
                    fname = 'tmp'
                else:
                    fname = erifile
                    _cleanup = False
                
                _read_eri = False
                if (type(eriread) is str):
                    fname = eriread
                    _read_eri = True
                    _cleanup = False

                _fname = fname

                _t1 = time.time()

                _mo_l = mo_coeff[:_nlrg, _nlrg:]
                _mo_s = mo_coeff[_nlrg:, _nlrg:]/(2*self.c0)

                if (frozen is not None):
                    _mem = self.nfrozen**4*16/1e9*2
                    if (_mem < 1.0):
                        if (self.verbose): print(f'Will now allocate {_mem*1000:.3f} MB disk space for the frozen core ERI tensor!')
                    else:
                        if (self.verbose): print(f'Will now allocate {_mem:.3f} GB disk space for the frozen core ERI tensor!')

                    fname = _fname + 'cccc.h5'
                    if (not _read_eri): h5_write(fname, (_frzc,_frzc,_frzc,_frzc))
                    with h5py.File(_fname+'cccc.h5', mode='r') as feri:
                        eri_fc = feri['mo_eri'][:].reshape((self.nfrozen, self.nfrozen, self.nfrozen, self.nfrozen))
                    self.e_frozen += 0.5*(np.einsum('iijj->',eri_fc)-np.einsum('ijji->',eri_fc))
                    if (_cleanup): os.remove(fname)
                    del eri_fc

                    _mem = 2*(self.nlrg*self.nfrozen)**2*16/1e9*2
                    if (_mem < 1.0):
                        if (self.verbose): print(f'Will now allocate {_mem*1000:.3f} MB disk space for the aacc and acca ERI tensor!')
                    else:
                        if (self.verbose): print(f'Will now allocate {_mem:.3f} GB disk space for the aacc and acca ERI tensor!')

                    fname = _fname + 'aacc.h5'
                    if (not _read_eri): h5_write(fname, (_actv,_actv,_frzc,_frzc))
                    with h5py.File(_fname+'aacc.h5', mode='r') as feri:
                        eri_fc_pqii = feri['mo_eri'][:].reshape((self.nlrg, self.nlrg, self.nfrozen, self.nfrozen))
                    self.dhf_hcore_mo = _dhf_hcore_mo[_actv,_actv].copy()
                    self.dhf_hcore_mo += np.einsum('pqii->pq', eri_fc_pqii)
                    if (_cleanup): os.remove(fname)
                    del eri_fc_pqii

                    fname = _fname + 'acca.h5'
                    if (not _read_eri): h5_write(fname, (_actv,_frzc,_frzc,_actv))
                    with h5py.File(_fname+'acca.h5', mode='r') as feri:
                        eri_fc_piiq = feri['mo_eri'][:].reshape((self.nlrg, self.nfrozen, self.nfrozen, self.nlrg))
                    self.dhf_hcore_mo -= np.einsum('piiq->pq', eri_fc_piiq)
                    if (_cleanup): os.remove(fname)
                    del eri_fc_piiq

                _mem = self.nlrg**4*16/1e9*2
                if (_mem < 1.0):
                    if (self.verbose): print(f'Will now allocate {_mem*1000:.3f} MB disk space for the AO ERI tensor!')
                else:
                    if (self.verbose): print(f'Will now allocate {_mem:.3f} GB disk space for the AO ERI tensor!')

                fname = _fname + '.h5'
                if (not _read_eri): h5_write(fname, (_actv,_actv,_actv,_actv))
                with h5py.File(fname, 'r') as feri:
                    _dhf_eri_mo_full = feri['mo_eri'][:].reshape((self.nlrg,self.nlrg,self.nlrg,self.nlrg))
                if (_cleanup): os.remove(fname)
                    
                _dhf_eri_full_asym = _dhf_eri_mo_full.swapaxes(1,2) - _dhf_eri_mo_full.swapaxes(1,2).swapaxes(2,3)
                _t2 = time.time()

                if (frozen is not None):
                    self.dhf_eri_full_asym = _dhf_eri_full_asym
                    del _dhf_eri_full_asym, _dhf_eri_mo_full
                else:
                    self.e_frozen = 0.0
                    self.dhf_eri_full_asym = _dhf_eri_full_asym
                    self.dhf_hcore_mo = _dhf_hcore_mo
            
        self.nuclear_repulsion += self.e_frozen
        if (self.verbose):
            print(f'\nTiming report')
            print(f'....integral retrieval:      {(_t1-_t0):15.7f} s')
            print(f'....integral transformation: {(_t2-_t1):15.7f} s')


In [57]:
mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
H 0 0 0
Cl 0 1.5 0
''',
    basis = 'cc-pvdz', spin=0, charge=0, symmetry=False, nucmod='G'
)
a = RelForte(mol, verbose=True, density_fitting=False, decontract=True)
a.run_dhf(transform=False)

              PySCF DHF interface              
Relativistic DHF Energy:        -461.5202725 Eh
PySCF RHF time:                    2.3010225 s
-----------------------------------------------


In [81]:
s,U = np.linalg.eigh(a.dhf.get_ovlp())
X = U @ np.diag(np.sqrt(1/s)) # The Lowdin canonicalization matrix
Xinv = np.diag(np.sqrt(s)) @ U.T.conj()

mo_coeff_on = Xinv@a.dhf.mo_coeff
cp = mo_coeff_on[a.nlrg:, :a.nlrg]
cl = mo_coeff_on[:a.nlrg, :a.nlrg]
Xtrans = np.linalg.solve(cp@cp.T.conj(), -cp@cl.T.conj())

W1 = np.block([[np.eye(a.nlrg), -Xtrans.T.conj()],[Xtrans, np.eye(a.nlrg)]])
W2 = np.block([[np.diag(np.sqrt(1/(1+np.diag(Xtrans.T.conj()@Xtrans)))) , np.zeros((a.nlrg, a.nlrg))],[np.zeros((a.nlrg, a.nlrg)), np.diag(np.sqrt(1/(1+np.diag(Xtrans@Xtrans.T.conj()))))]])
Utrans = W1@W2

coeff_2c = (Utrans.T.conj()@mo_coeff_on)[:a.nlrg, a.nlrg:]

In [82]:
coeff_2c.shape

(96, 96)

In [96]:
eri = np.einsum('pi,qj,pqrs,rk,sl->ijkl', np.conj(coeff_2c),coeff_2c,a.mol.intor('int2e_spinor'),np.conj(coeff_2c),coeff_2c,optimize=True)
eri_asym = eri.swapaxes(1,2) - eri.swapaxes(1,2).swapaxes(2,3)

In [97]:
a.dhf.mo_energy[a.nlrg:a.nlrg+a.nocc].sum() - 0.5*np.einsum('ijij->', eri_asym[:a.nocc,:a.nocc,:a.nocc,:a.nocc]) + a.nuclear_repulsion

(-437.37915419447995+2.1418070058341255e-15j)